## 1 import packages

In [1]:
#!/usr/bin/env python
# coding=utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# sys.path.append("/search/odin/yangyuran/program/Anaconda3/envs/tensorflow/lib/python3.6/site-packages/")


import datetime
import os

import numpy as np
import math
import tensorflow as tf
import data_process
import model

/search/odin/yangyuran/program/Anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/search/odin/yangyuran/program/Anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 2 指定参数

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

N_CLASSES = 2
IMG_W = 256  # resize the image, if the input image is too large, training will be very slow.
IMG_H = 256
IMG_C = 3
RATIO = 0.2  # take 20% of dataset as validation data
BATCH_SIZE = 32
CAPACITY = 100 + 3 * BATCH_SIZE
MAX_STEP = 5000  # with current parameters, it is suggested to use MAX_STEP>10k
learning_rate = 0.0001  # with current parameters, it is suggested to use learning rate<0.0001

train_dir = '/search/odin/xudongmei/working/datasets/bak_crop/train_split/'
test_dir = '/search/odin/xudongmei/working/datasets/bak_crop/test_split/'
logs_train_dir = './logs/bak_crop/train/batch_size=32/lr=0.0001/'
logs_val_dir = './logs/bak_crop/val/batch_size=32/lr=0.0001/'


## 3. training()

In [3]:
def run_training():
    with tf.Graph().as_default():
        train, train_label, val, val_label = data_process.get_files(train_dir, RATIO)

        train_batch, train_label_batch = data_process.get_batch(train,
                                                                train_label,
                                                                IMG_W,
                                                                IMG_H,
                                                                BATCH_SIZE,
                                                                CAPACITY)
        val_batch, val_label_batch = data_process.get_batch(val,
                                                            val_label,
                                                            IMG_W,
                                                            IMG_H,
                                                            BATCH_SIZE,
                                                            CAPACITY)

        x = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMG_W, IMG_H, 3])
        y_ = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
        keep_prob = tf.placeholder(tf.float32)

        logits = model.inference(x, BATCH_SIZE, N_CLASSES)
        loss = model.losses(logits, y_)
        acc = model.evaluation(logits, y_)
        train_op = model.trainning(loss, learning_rate)

        with tf.Session() as sess:
            saver = tf.train.Saver()
            sess.run(tf.global_variables_initializer())
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            summary_op = tf.summary.merge_all()
            train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)
            # val_writer = tf.summary.FileWriter(logs_val_dir, sess.graph)
            val_writer = tf.summary.FileWriter(logs_val_dir)

            try:
                for step in np.arange(MAX_STEP):
                    if coord.should_stop():
                        break

                    tra_images, tra_labels = sess.run([train_batch, train_label_batch])
                    # print("tra_images's type: ", type(tra_images))
                    # print("tra_images.dtype: ", tra_images.dtype)
                    #
                    # print("tra_images's shape", tra_images.shape)
                    _, tra_loss, tra_acc = sess.run([train_op, loss, acc],
                                                    feed_dict={x: tra_images, y_: tra_labels, keep_prob: 0.5})
                    if step % 50 == 0:
                        print('Step %d, train loss = %.2f, train accuracy = %.2f%%' % (step, tra_loss, tra_acc * 100.0))
                        summary_str = sess.run(summary_op, feed_dict={x: tra_images, y_: tra_labels, keep_prob: 0.5})
                        train_writer.add_summary(summary_str, step)

                    if step % 200 == 0 or (step + 1) == MAX_STEP:
                        val_images, val_labels = sess.run([val_batch, val_label_batch])
                        val_loss, val_acc = sess.run([loss, acc],
                                                     feed_dict={x: val_images, y_: val_labels, keep_prob: 0.5})
                        print('**  Step %d, val loss = %.2f, val accuracy = %.2f%%  **' % (
                            step, val_loss, val_acc * 100.0))
                        summary_str = sess.run(summary_op, feed_dict={x: val_images, y_: val_labels, keep_prob: 0.5})
                        val_writer.add_summary(summary_str, step)

                    if step % 2000 == 0 or (step + 1) == MAX_STEP:
                        checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
                        saver.save(sess, checkpoint_path, global_step=step)

            except tf.errors.OutOfRangeError:
                print('Done training -- epoch limit reached')
            finally:
                coord.request_stop()
            coord.join(threads)


In [4]:
begin = datetime.datetime.now()
run_training()
end = datetime.datetime.now()
print("Training time is: ", end - begin)

There are 16 baidu
There are 16 sogou
Step 0, train loss = 0.69, train accuracy = 40.62%
**  Step 0, val loss = 0.69, val accuracy = 56.25%  **
Step 50, train loss = 0.00, train accuracy = 100.00%
Step 100, train loss = 0.00, train accuracy = 100.00%
Step 150, train loss = 0.00, train accuracy = 100.00%
Step 200, train loss = 0.00, train accuracy = 100.00%
**  Step 200, val loss = 3.11, val accuracy = 40.62%  **
Step 250, train loss = 0.00, train accuracy = 100.00%
Step 300, train loss = 0.00, train accuracy = 100.00%
Step 350, train loss = 0.00, train accuracy = 100.00%
Step 400, train loss = 0.00, train accuracy = 100.00%
**  Step 400, val loss = 3.73, val accuracy = 34.38%  **
Step 450, train loss = 0.00, train accuracy = 100.00%
Step 500, train loss = 0.00, train accuracy = 100.00%
Step 550, train loss = 0.00, train accuracy = 100.00%
Step 600, train loss = 0.00, train accuracy = 100.00%
**  Step 600, val loss = 3.04, val accuracy = 59.38%  **
Step 650, train loss = 0.00, train acc

## 4 evalution

In [7]:
def evaluate():
    with tf.Graph().as_default():
        test, test_label = data_process.get_files(test_dir, 0, False)
        test_batch, test_label_batch = data_process.get_batch(test,
                                                              test_label,
                                                              IMG_W,
                                                              IMG_H,
                                                              BATCH_SIZE,
                                                              CAPACITY)

        with tf.Session() as sess:
            x = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMG_W, IMG_H, IMG_C])
            y_ = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
            keep_prob = tf.placeholder(tf.float32)

            logits = model.inference(x, BATCH_SIZE, N_CLASSES)
            logits = tf.nn.softmax(logits)
            acc = model.evaluation(logits, y_)

            saver = tf.train.Saver()
            # 模型恢复
            print("Reading checkpoints...")
            ckpt = tf.train.get_checkpoint_state(logs_train_dir)
            if ckpt and ckpt.model_checkpoint_path:
                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                saver.restore(sess, ckpt.model_checkpoint_path)
                print('Loading success, global_step is %s' % global_step)
            else:
                print('No checkpoint file found')
                return

            # Start the queue runners.
            coord = tf.train.Coordinator()
            threads = []
            try:
                for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
                    threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                                     start=True))

                num_iter = int(math.ceil(len(test) / BATCH_SIZE))
                total_acc = 0  # Counts the total predictions.
                total_sample_count = num_iter * BATCH_SIZE
                step = 0
                # print(num_iter)

                test_images, test_labels = sess.run([test_batch, test_label_batch])
                # print("test_images dtype: ", test_images.dtype)  # test_images dtype:  float32
                # print("test_images type: ", type(test_images))  # test_images type:  <class 'numpy.ndarray'>

                while step < num_iter and not coord.should_stop():
                    # 每次预测的准确率
                    predictions, accuracies = sess.run([logits, acc],
                                                       feed_dict={x: test_images, y_: test_labels, keep_prob: 1.0})
                    # 计算总的准确率
                    total_acc += np.sum(accuracies)
                    # print(total_acc)
                    step += 1

                print(len(predictions))
                # Compute precision @ 1.
                avg_accuracies = total_acc / num_iter
                print('%s: precision @ 1 = %.3f' % (datetime.datetime.now(), avg_accuracies))

                """将预测结果写入文件"""
                print('Begin to write submission file ..')
                f_submit = open('predict_1.csv', 'w')
                f_submit.write('true_label,baidu,sogou\n')
                for i, image_name in enumerate(test_labels):
                    pred = ['%.2f' % p for p in predictions[i, :]]
                    f_submit.write('%s,%s\n' % (image_name, ','.join(pred)))

                f_submit.close()

                print('Submission file successfully generated!')

            except Exception as e:  # pylint: disable=broad-except
                coord.request_stop(e)
            finally:
                coord.request_stop()
            coord.join(threads)


In [8]:
evaluate()

There are 5 baidu
There are 5 sogou
Reading checkpoints...
INFO:tensorflow:Restoring parameters from ./logs/bak_crop/train/batch_size=32/lr=0.0001/model.ckpt-4999
Loading success, global_step is 4999
32
2018-05-24 18:22:18.130482: precision @ 1 = 0.375
Begin to write submission file ..
Submission file successfully generated!
